# Food supply of world regions

This notebook explains how we generated the data on food supply of world regions, shown in this data insight.

You can run this notebook on [Google Colab](https://colab.research.google.com/github/owid/etl/blob/docs-technical-publication-agriculturalindicators/docs/analyses/food_supply_of_world_regions/food_supply_of_world_regions.ipynb), or download the folder to run it on your computer: [this notebook](food_supply_of_world_regions.ipynb) | [full folder](https://catalog.owid.io/analyses/food-supply-of-world-regions.zip) (including notebook, input data and results).

* TODO: Add link to data insight once published.
* TODO: Replace branch with master before merging.

## Initial setup
### Install and import requirements

To be able to run the code, there are some libraries you may need to install.

* TODO: Decide if we need to import modules (possibly a subset of FAOSTAT data), and update link to point to master.

In [3]:
# Install dependencies
%pip install -q pandas==2.2.2 plotly==6.3.0 kaleido==1.1.0
# Fetch adjacent modules and files if running in Colab
import urllib.request, os, sys
if "google.colab" in sys.modules:
    for file in ["resouces/faostat_qcl_selected_data.csv"]:
        os.makedirs(os.path.dirname(file) or ".", exist_ok=True)
        urllib.request.urlretrieve(f"https://raw.githubusercontent.com/owid/etl/blob/docs-technical-publication-agriculturalindicators/docs/analyses/food_supply_of_world_regions/{file}", file)

/Users/prosado/Documents/owid/repos/etl/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries.

In [1]:
import pandas as pd
import plotly.express as px

### Set common variables

There are some parameters you need to set before the analysis can be run:
- <code>VERBOSE</code>: If True, the script will output some information on its progress and intermediary results

In [2]:
# True to re-run the code; False to load the exported data files.
USE_SAVED_RESULTS = False

# True to overwrite the output data files.
OVERWRITE = True

## Data processing
### Get deaths for each cause of death

This data comes from two files, both downloaded from the CDC Wonder database "[Underlying Cause of Death Data](https://wonder.cdc.gov/deaths-by-underlying-cause.html)" and hosted on the Our World in Data R2 storage.

- The leading causes file is the result when selecting "Group results by" - "15 leading causes of death" and selecting the year 2023
- The external factors file is the result when selecting "Group result by" - "Cause of death", selecting the year 2023 and only selecting cause of death "V01-Y89 (External causes of mobidity and mortality)"